In [19]:
import random
import numpy as np

In [67]:
def G1(x):
  return 5 * (x[0] + x[1] + x[2] + x[3]) - 5 * sum(x[i]**2 for i in range(4)) - sum(x[i] for i in range(4, 13))

def G1_bounds_check(x):
    for i in range(9):
        if not (0 <= x[i] <= 1):
            return False
    for i in range(9, 12):
        if not (0 <= x[i] <= 100):
            return False
    if not (0 <= x[12] <= 1):
        return False
    return True

def constraint_1(x):
    return 2 * x[0] + 2 * x[1] + x[9] + x[10] <= 10

def constraint_2(x):
    return 2 * x[0] + 2 * x[2] + x[9] + x[11] <= 10

def constraint_3(x):
    return 2 * x[0] + 2 * x[3] + x[10] + x[11] <= 10

def constraint_4(x):
    return -8 * x[0] + x[12] <= 0

def constraint_5(x):
    return -8 * x[1] + x[9] <= 0

def constraint_6(x):
    return -8 * x[2] + x[10] <= 0

def constraint_7(x):
    return -8 * x[3] + x[11] <= 0

def constraint_8(x):
    return -2 * x[4] - x[5] + x[9] <= 0

def constraint_9(x):
    return -2 * x[6] - x[7] + x[10] <= 0

def constraint_10(x):
    return -2 * x[8] - x[12] + x[11] <= 0

# Define bounds for each variable in x
G1_bounds = [
    (0, 1),  # x1
    (0, 1),  # x2
    (0, 1),  # x3
    (0, 1),  # x4
    (0, 1),  # x5
    (0, 1),  # x6
    (0, 1),  # x7
    (0, 1),  # x8
    (0, 1),  # x9
    (0, 100),  # x10
    (0, 100),  # x11
    (0, 100),  # x12
    (0, 1)   # x13
]


In [71]:
def G4(x):
    return 3.5578574 * x[0]**2 + 0.8356891 * x[0] * x[4] + 37.293239 * x[0] - 40792.141

def G4_bounds_check(x, bounds):
    for i, (lower, upper) in enumerate(bounds):
        if not (lower <= x[i] <= upper):
            return False
    return True

def G4_constraint_1(x):
    value = 85.334407 + 0.0056858 * x[1] * x[4] + 0.0006262 * x[0] * x[3] - 0.0022053 * x[2] * x[4]
    return 0 <= value <= 92

def G4_constraint_2(x):
    value = 80.51249 + 0.0071317 * x[1] * x[4] + 0.0029955 * x[0] * x[1] + 0.0021813 * x[2]**2
    return 90 <= value <= 110

def G4_constraint_3(x):
    value = 9.300961 + 0.0047026 * x[2] * x[4] + 0.0012547 * x[0] * x[2] + 0.0019085 * x[2] * x[3]
    return 20 <= value <= 25


# Bounds for each variable in x
G4_bounds = [
    (78, 102),  # x1
    (33, 45),   # x2
    (27, 45),   # x3
    (27, 45),   # x4
    (27, 45)    # x5
]

In [75]:
def G5(x):
    return 3 * x[0] + 0.000001 * x[0]**3 + 2 * x[1] + (0.000002 / 3) * x[2]**3

def G5_bounds_check(x, bounds):
    for i, (lower, upper) in enumerate(bounds):
        if not (lower <= x[i] <= upper):
            return False
    return True

def G5_constraint_1(x):
    return x[3] - x[2] + 0.55 >= 0

def G5_constraint_2(x):
    return x[2] - x[3] + 0.55 >= 0

def G5_constraint_3(x):
    return np.isclose(1000 * np.sin(x[1] - x[0] - 0.25) + 1000 * np.sin(x[3] - x[2] - 0.25) + 894.8 - x[0], 0, atol=1e-6)

def G5_constraint_4(x):
    return np.isclose(1000 * np.sin(x[0] - x[1] - 0.25) + 1000 * np.sin(x[2] - x[3] - 0.25) + 894.8 - x[1], 0, atol=1e-6)

def G5_constraint_5(x):
    return np.isclose(1000 * np.sin(x[3] - x[0] - 0.25) + 1000 * np.sin(x[2] - x[1] - 0.25) + 1294.8, 0, atol=1e-6)

# Bounds for each variable in x
G5_bounds = [
    (0, 1200),   # x1
    (0, 1200),   # x2
    (-0.55, 0.55),  # x3
    (-0.55, 0.55)   # x4
]


In [77]:
def G6(x):
    return (x[0] - 10)**3 + (x[1] - 20)**3

def G6_bounds_check(x, bounds):
    for i, (lower, upper) in enumerate(bounds):
        if not (lower <= x[i] <= upper):
            return False
    return True

def G6_constraint_1(x):
    return (x[0] - 5)**2 + (x[1] - 5)**2 - 100 >= 0

def G6_constraint_2(x):
    return -(x[0] - 6)**2 - (x[1] - 5)**2 + 82.81 >= 0

# Bounds for each variable in x
G6_bounds = [
    (13, 100),  # x1
    (0, 100)    # x2
]

In [5]:
PC_REAL = 0.9
PM_REAL = 0.01
N_C = 20

In [61]:
def clip_to_bounds(x, bounds):
    return np.array([np.clip(x[i], lower, upper) for i, (lower, upper) in enumerate(bounds)])

In [58]:
# Real encoding crossover (SBX) and mutation
def sbx_crossover(p1, p2, nc):
  child1, child2 = np.zeros_like(p1), np.zeros_like(p2)
  for i in range(len(p1)):
    u = np.random.rand()
    if u <= 0.5:
      beta = (2 * u)**(1 / (nc + 1))
    else:
      beta = (1 / (2 * (1 - u)))**(1 / (nc + 1))
    child1[i] = 0.5 * ((1 + beta) * p1[i] + (1 - beta) * p2[i])
    child2[i] = 0.5 * ((1 - beta) * p1[i] + (1 + beta) * p2[i])
  return child1, child2

# Polynomial mutation
def poly_mutation(chrom, nm=20):
  x_max = max(chrom)
  x_min = min(chrom)
  for i, alelle in enumerate(chrom):
    r = np.random.rand()
    if r <= 0.5:
      delta_i = ((2*r)**(1/(nm+1)))-1
    else:
      delta_i = 1-((2*(1-r))**(1/(nm+1)))
    alelle += delta_i*(x_max - x_min)
    chrom[i] = alelle
  return chrom

In [64]:
# Genetic Algorithm framework
def genetic_algorithm(fitness_function, bounds, dim, ineq_constraints, eq_constraints, con1=4, con2=3, beta=1, gamma=2, f_thresh=0.5, delta=0.1, n_generations=100, pop_size=100, stop_threshold=1e-6, stagnation_generations=5, verbose=False):
    # Create initial population as real values
    population = [
    np.array([np.random.uniform(lower, upper) for lower, upper in bounds])
    for _ in range(pop_size)
    ]

    
    if verbose:
        print(f'Original population: {population}')

    best_fitness_per_gen = []
    
    for gen in range(n_generations):
        # Clip the values of the real population to ensure they stay within bounds
        real_population = [clip_to_bounds(ind, bounds) for ind in population]

        if verbose:
            print(f'Clipped population (generation {gen + 1}): {real_population}')

        # Evaluate fitness using penalty function
        fitness = []
        count_feasible = 0
        for individual in real_population:
            G_i, H_i = [], []
            is_feasible = True  # Track if the individual is feasible
            for constraint in ineq_constraints:
                violation = max(0, constraint(individual))**beta
                G_i.append(violation)
                if violation > 0:
                    is_feasible = False  # If any violation occurs, mark as infeasible
            for constraint in eq_constraints:
                violation = np.abs(constraint(individual))**gamma
                H_i.append(violation)
                if violation > 0:
                    is_feasible = False
            if is_feasible:
                count_feasible += 1
            
            violation = con1 * np.sum(G_i) + con2 * np.sum(H_i)
            fitness.append(fitness_function(individual) + violation)
    
        new_population = []
        copy_population = [list(ind) for ind in real_population]
        for _ in range(pop_size // 2):
            p1 = random.choice(copy_population)
            copy_population.pop(copy_population.index(p1))
            p2 = random.choice(copy_population)
            if np.random.rand() < PC_REAL:
                c1, c2 = sbx_crossover(p1, p2, N_C)
            else:
                c1, c2 = p1, p2
            if np.random.rand() < PM_REAL:
                c1 = poly_mutation(c1)
                c2 = poly_mutation(c2)
            
            c1 = clip_to_bounds(c1, bounds)
            c2 = clip_to_bounds(c2, bounds)
            
            new_population.extend([c1, c2])
        population = new_population

        # Track best fitness
        best_fitness = max(fitness)
        best_solution_index = fitness.index(best_fitness)
        best_solution = real_population[best_solution_index]
        best_fitness_per_gen.append(best_fitness)

        f_ratio = count_feasible/len(real_population)
        if f_ratio < f_thresh:
            con1 *= 1 - delta
            con2 *= 1 - delta
        else:
            con1 *= 1 + delta
            con2 *= 1 + delta

        if verbose:    
            print(f"Generation {gen + 1}, Best Fitness: {best_fitness}")

        # Check for stagnation (early stopping)
        if gen >= stagnation_generations:
            recent_fitness = best_fitness_per_gen[-stagnation_generations:]
            fitness_change = max(recent_fitness) - min(recent_fitness)

            if fitness_change < stop_threshold and verbose:
                print(f"Early stopping at generation {gen + 1} due to insufficient improvement.")
                break

    return best_fitness_per_gen, best_solution

In [ ]:
solutions = []
fitness_per_gen = []
for i in range(30):
  fit_per_gen, sol = genetic_algorithm(G1, 
                                      G1_bounds, 
                                      13, 
                                      ineq_constraints=[
                                        constraint_1,
                                        constraint_2,
                                        constraint_3,
                                        constraint_4,
                                        constraint_5,
                                        constraint_6,
                                        constraint_7,
                                        constraint_8,
                                        constraint_9,
                                        constraint_10
                                      ],
                                      eq_constraints=[]                                      )
  fitness_per_gen.append(fit_per_gen)
  solutions.append(sol)
  is_feasible = G1_bounds_check(sol)
  print(f'Experiment {i + 1}, solution is {sol} and feasability is {is_feasible}')

mean = np.mean(solutions)
std_dev = np.std(solutions)

print(f'Mean: {mean}, Standard Deviation: {std_dev}')

In [ ]:
import matplotlib.pyplot as plt
# Assuming fitness_per_gen is a list of lists with 30 runs
# Each sub-list contains fitness values across generations for one run
# Example: fitness_per_gen = [[gen1_run1, gen2_run1, ...], [gen1_run2, gen2_run2, ...], ...]

# Generate some dummy data for illustration (replace this with your actual fitness_per_gen)
# Example: 30 runs, 100 generations each
num_runs = 30
num_generations = 100
fitness_per_gen = [np.random.uniform(0, 1, num_generations).cumsum() for _ in range(num_runs)]

# Plot all runs in a single convergence plot
plt.figure(figsize=(10, 6))

for run in fitness_per_gen:
    plt.plot(run, alpha=0.5)  # Plot each run with a slight transparency

# Adding labels and title
plt.xlabel("Generation")
plt.ylabel("G1 Fitness")
plt.title("Convergence Plot for 30 Runs")
plt.grid(True)
plt.show()


In [72]:
solutions = []
fitness_per_gen = []
for i in range(30):
  fit_per_gen, sol = genetic_algorithm(G4, 
                                      G4_bounds, 
                                      5, 
                                      ineq_constraints=[
                                        G4_constraint_1,
                                        G4_constraint_2,
                                        G4_constraint_3
                                      ],
                                      eq_constraints=[]                                      )
  fitness_per_gen.append(fit_per_gen)
  solutions.append(sol)
  is_feasible = G4_bounds_check(sol, G4_bounds)
  print(f'Experiment {i + 1}, solution is {sol} and feasability is {is_feasible}')

mean = np.mean(solutions)
std_dev = np.std(solutions)

print(f'Mean: {mean}, Standard Deviation: {std_dev}')

Experiment 1, solution is [99.50558112 36.74583987 29.70605873 32.99888369 35.52676366] and feasability is True
Experiment 2, solution is [101.36704643  41.76094237  29.86811352  31.48637374  30.43427253] and feasability is True


KeyboardInterrupt: 

In [ ]:
# Assuming fitness_per_gen is a list of lists with 30 runs
# Each sub-list contains fitness values across generations for one run
# Example: fitness_per_gen = [[gen1_run1, gen2_run1, ...], [gen1_run2, gen2_run2, ...], ...]

# Generate some dummy data for illustration (replace this with your actual fitness_per_gen)
# Example: 30 runs, 100 generations each
num_runs = 30
num_generations = 100
fitness_per_gen = [np.random.uniform(0, 1, num_generations).cumsum() for _ in range(num_runs)]

# Plot all runs in a single convergence plot
plt.figure(figsize=(10, 6))

for run in fitness_per_gen:
    plt.plot(run, alpha=0.5)  # Plot each run with a slight transparency

# Adding labels and title
plt.xlabel("Generation")
plt.ylabel("G4 Fitness")
plt.title("Convergence Plot for 30 Runs")
plt.grid(True)
plt.show()

In [76]:
solutions = []
fitness_per_gen = []
for i in range(30):
  fit_per_gen, sol = genetic_algorithm(G5, 
                                      G5_bounds, 
                                      4, 
                                      ineq_constraints=[
                                        G5_constraint_1,
                                        G5_constraint_2,
                                        G5_constraint_3,
                                        G5_constraint_4,
                                        G5_constraint_5
                                      ],
                                      eq_constraints=[]                                      )
  fitness_per_gen.append(fit_per_gen)
  solutions.append(sol)
  is_feasible = G5_bounds_check(sol, G5_bounds)
  print(f'Experiment {i + 1}, solution is {sol} and feasability is {is_feasible}')

mean = np.mean(solutions)
std_dev = np.std(solutions)

print(f'Mean: {mean}, Standard Deviation: {std_dev}')

Experiment 1, solution is [ 1.19885888e+03  6.28870088e+02 -1.03547317e-02  3.09999459e-01] and feasability is True


KeyboardInterrupt: 

In [ ]:
# Assuming fitness_per_gen is a list of lists with 30 runs
# Each sub-list contains fitness values across generations for one run
# Example: fitness_per_gen = [[gen1_run1, gen2_run1, ...], [gen1_run2, gen2_run2, ...], ...]

# Generate some dummy data for illustration (replace this with your actual fitness_per_gen)
# Example: 30 runs, 100 generations each
num_runs = 30
num_generations = 100
fitness_per_gen = [np.random.uniform(0, 1, num_generations).cumsum() for _ in range(num_runs)]

# Plot all runs in a single convergence plot
plt.figure(figsize=(10, 6))

for run in fitness_per_gen:
    plt.plot(run, alpha=0.5)  # Plot each run with a slight transparency

# Adding labels and title
plt.xlabel("Generation")
plt.ylabel("G5 Fitness")
plt.title("Convergence Plot for 30 Runs")
plt.grid(True)
plt.show()

In [78]:
solutions = []
fitness_per_gen = []
for i in range(30):
  fit_per_gen, sol = genetic_algorithm(G6, 
                                      G6_bounds, 
                                      2, 
                                      ineq_constraints=[
                                        G6_constraint_1,
                                        G6_constraint_2
                                      ],
                                      eq_constraints=[]                                      )
  fitness_per_gen.append(fit_per_gen)
  solutions.append(sol)
  is_feasible = G6_bounds_check(sol, G6_bounds)
  print(f'Experiment {i + 1}, solution is {sol} and feasability is {is_feasible}')

mean = np.mean(solutions)
std_dev = np.std(solutions)

print(f'Mean: {mean}, Standard Deviation: {std_dev}')

Experiment 1, solution is [64.49636864 99.16876338] and feasability is True
Experiment 2, solution is [94.43200405 99.75766662] and feasability is True
Experiment 3, solution is [77.3670688  98.87304004] and feasability is True


KeyboardInterrupt: 

In [ ]:
# Assuming fitness_per_gen is a list of lists with 30 runs
# Each sub-list contains fitness values across generations for one run
# Example: fitness_per_gen = [[gen1_run1, gen2_run1, ...], [gen1_run2, gen2_run2, ...], ...]

# Generate some dummy data for illustration (replace this with your actual fitness_per_gen)
# Example: 30 runs, 100 generations each
num_runs = 30
num_generations = 100
fitness_per_gen = [np.random.uniform(0, 1, num_generations).cumsum() for _ in range(num_runs)]

# Plot all runs in a single convergence plot
plt.figure(figsize=(10, 6))

for run in fitness_per_gen:
    plt.plot(run, alpha=0.5)  # Plot each run with a slight transparency

# Adding labels and title
plt.xlabel("Generation")
plt.ylabel("G5 Fitness")
plt.title("Convergence Plot for 30 Runs")
plt.grid(True)
plt.show()